<a href="https://colab.research.google.com/github/Prabzmihi/DDODM/blob/main/Microalloyed_Steels_MO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV

###Question 1 - Answer (Data Preprocessing)

In [9]:
#Access files from Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

data = pd.read_excel("/content/gdrive/MyDrive/TIES5830/Metal_Data.xlsx", header=0)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [10]:
print(data.head())

    Nb     Ti      V  DATA NO Paper no.       C    Si    Mn     P      S  ...  \
0  0.0  0.000  0.000        5        E19  0.140  0.35  1.56  0.01  0.015  ...   
1  0.0  0.000  0.096        6        E19  0.150  0.31  1.41  0.01  0.015  ...   
2  0.0  0.075  0.000        9        h 1  0.054  0.03  0.54  0.00  0.000  ...   
3  0.0  0.092  0.000       10        h 1  0.061  0.03  0.56  0.00  0.000  ...   
4  0.0  0.092  0.000       11        h 1  0.055  0.03  0.55  0.00  0.000  ...   

    Cr   Ce   Cu   Ca   Zr    O  Sn  YS(Mpa)  UTS(Mpa)   %EL  
0  0.0  0.0  0.0  0.0  0.0  0.0   0    272.0     470.0  30.0  
1  0.0  0.0  0.0  0.0  0.0  0.0   0    378.0     579.0  23.0  
2  0.0  0.0  0.0  0.0  0.0  0.0   0    412.0       NaN   NaN  
3  0.0  0.0  0.0  0.0  0.0  0.0   0    410.0       NaN   NaN  
4  0.0  0.0  0.0  0.0  0.0  0.0   0    395.0       NaN   NaN  

[5 rows x 25 columns]


In [11]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 739 entries, 0 to 738
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Nb          739 non-null    float64
 1   Ti          739 non-null    float64
 2   V           739 non-null    float64
 3   DATA NO     739 non-null    int64  
 4   Paper no.   739 non-null    object 
 5   C           739 non-null    float64
 6   Si          739 non-null    float64
 7   Mn          739 non-null    float64
 8   P           739 non-null    float64
 9   S           739 non-null    float64
 10  Mo          739 non-null    float64
 11  Ni          739 non-null    float64
 12  Al          739 non-null    float64
 13  N           739 non-null    float64
 14  B           739 non-null    float64
 15  Cr          739 non-null    float64
 16  Ce          739 non-null    float64
 17  Cu          739 non-null    float64
 18  Ca          739 non-null    float64
 19  Zr          739 non-null    f

In [12]:
print(data.describe())

               Nb          Ti           V     DATA NO           C          Si  \
count  739.000000  739.000000  739.000000  739.000000  739.000000  739.000000   
mean     0.014490    0.007555    0.025546  254.529093    0.112910    0.255689   
std      0.024305    0.020186    0.055370  143.193390    0.088669    0.132574   
min      0.000000    0.000000    0.000000    1.000000    0.001200    0.000000   
25%      0.000000    0.000000    0.000000  118.000000    0.069000    0.180000   
50%      0.005000    0.000000    0.000000  274.000000    0.090000    0.280000   
75%      0.025000    0.009000    0.030000  378.500000    0.150000    0.350000   
max      0.450000    0.310000    0.800000  505.000000    1.030000    0.600000   

               Mn           P           S          Mo  ...          Cr  \
count  739.000000  739.000000  739.000000  739.000000  ...  739.000000   
mean     1.226204    0.013127    0.007229    0.026169  ...    0.046482   
std      0.347358    0.033446    0.007539    0.1

In [13]:
alloying_elements = ["Nb", "Ti", "V", "C", "Si", "Mn", "P", "S", "Mo", "Ni", "Al", "N", "B", "Cr", "Ce", "Cu", "Ca", "Zr", "O", "Sn"]

In [14]:
df_ys = data[data["YS(Mpa)"].notna()]
selected_columns_ys = alloying_elements + ["YS(Mpa)"]
df_ys = df_ys[selected_columns_ys]

print(df_ys)

train_ys, test_ys = train_test_split(df_ys, test_size=0.2, random_state=42)

        Nb     Ti      V      C    Si    Mn      P      S   Mo   Ni  ...  \
0    0.000  0.000  0.000  0.140  0.35  1.56  0.010  0.015  0.0  0.0  ...   
1    0.000  0.000  0.096  0.150  0.31  1.41  0.010  0.015  0.0  0.0  ...   
2    0.000  0.075  0.000  0.054  0.03  0.54  0.000  0.000  0.0  0.0  ...   
3    0.000  0.092  0.000  0.061  0.03  0.56  0.000  0.000  0.0  0.0  ...   
4    0.000  0.092  0.000  0.055  0.03  0.55  0.000  0.000  0.0  0.0  ...   
..     ...    ...    ...    ...   ...   ...    ...    ...  ...  ...  ...   
734  0.086  0.047  0.000  0.070  0.14  1.35  0.009  0.003  0.0  0.0  ...   
735  0.100  0.000  0.000  0.120  0.14  1.15  0.010  0.010  0.0  0.0  ...   
736  0.220  0.055  0.000  0.080  0.00  0.79  0.000  0.000  0.0  0.0  ...   
737  0.450  0.310  0.000  0.100  0.08  1.47  0.000  0.000  0.0  0.0  ...   
738  0.080  0.070  0.000  0.060  0.00  1.50  0.000  0.000  0.0  0.0  ...   

          N    B   Cr   Ce   Cu   Ca   Zr    O  Sn  YS(Mpa)  
0    0.0090  0.0  0.0  0.

In [15]:
df_uts = data[data["UTS(Mpa)"].notna()]
selected_columns_uts = alloying_elements + ["UTS(Mpa)"]
df_uts = df_uts[selected_columns_uts]

print(df_uts)

train_uts, test_uts = train_test_split(df_uts, test_size=0.2, random_state=42)

        Nb     Ti      V      C    Si    Mn      P      S    Mo    Ni  ...  \
0    0.000  0.000  0.000  0.140  0.35  1.56  0.010  0.015  0.00  0.00  ...   
1    0.000  0.000  0.096  0.150  0.31  1.41  0.010  0.015  0.00  0.00  ...   
53   0.000  0.000  0.000  0.207  0.22  1.28  0.001  0.006  0.52  0.61  ...   
54   0.000  0.000  0.000  0.203  0.23  1.34  0.001  0.015  0.53  0.50  ...   
55   0.000  0.000  0.000  0.200  0.22  1.29  0.001  0.027  0.51  0.62  ...   
..     ...    ...    ...    ...   ...   ...    ...    ...   ...   ...  ...   
733  0.086  0.011  0.000  0.030  0.16  1.49  0.013  0.001  0.00  0.16  ...   
734  0.086  0.047  0.000  0.070  0.14  1.35  0.009  0.003  0.00  0.00  ...   
735  0.100  0.000  0.000  0.120  0.14  1.15  0.010  0.010  0.00  0.00  ...   
737  0.450  0.310  0.000  0.100  0.08  1.47  0.000  0.000  0.00  0.00  ...   
738  0.080  0.070  0.000  0.060  0.00  1.50  0.000  0.000  0.00  0.00  ...   

         N    B    Cr   Ce    Cu   Ca   Zr    O  Sn  UTS(Mpa)  

In [16]:
df_elon = data[data["%EL"].notna()]
selected_columns_elon = alloying_elements + ["%EL"]
df_elon = df_elon[selected_columns_elon]

print(df_elon)

train_elon, test_elon = train_test_split(df_elon, test_size=0.2, random_state=42)

        Nb     Ti      V      C    Si    Mn      P      S    Mo    Ni  ...  \
0    0.000  0.000  0.000  0.140  0.35  1.56  0.010  0.015  0.00  0.00  ...   
1    0.000  0.000  0.096  0.150  0.31  1.41  0.010  0.015  0.00  0.00  ...   
44   0.000  0.000  0.000  0.210  0.28  1.37  0.033  0.021  0.00  0.00  ...   
45   0.000  0.000  0.000  0.210  0.28  1.37  0.033  0.021  0.00  0.00  ...   
46   0.000  0.000  0.000  0.210  0.28  1.37  0.033  0.021  0.00  0.00  ...   
..     ...    ...    ...    ...   ...   ...    ...    ...   ...   ...  ...   
731  0.080  0.075  0.000  0.060  0.00  1.50  0.000  0.000  0.15  0.00  ...   
732  0.080  0.070  0.000  0.006  0.00  1.50  0.000  0.000  0.00  0.00  ...   
733  0.086  0.011  0.000  0.030  0.16  1.49  0.013  0.001  0.00  0.16  ...   
735  0.100  0.000  0.000  0.120  0.14  1.15  0.010  0.010  0.00  0.00  ...   
737  0.450  0.310  0.000  0.100  0.08  1.47  0.000  0.000  0.00  0.00  ...   

          N      B    Cr   Ce    Cu   Ca   Zr    O  Sn   %EL  


###Question 3 - Answer (Optimize the hyperparameters)

In [17]:
rf = RandomForestRegressor()

param_dist = {
    'n_estimators': [100, 200, 300],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 4, 6],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

In [18]:
# Perform RandomizedSearch for YS
rf_random_ys = RandomizedSearchCV(estimator=rf, param_distributions=param_dist,
                               n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)

# Fit the model with the training data
rf_random_ys.fit(train_ys.drop("YS(Mpa)", axis=1), train_ys["YS(Mpa)"])
best_params_ys = rf_random_ys.best_params_

print("Best hyperparameters:", best_params_ys)

best_model = rf_random_ys.best_estimator_
best_score = rf_random_ys.best_score_
print("Best score:", best_score)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best hyperparameters: {'n_estimators': 100, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': True}
Best score: 0.7014873749940714


In [19]:
# Perform RandomizedSearch for UTS
rf_random_uts = RandomizedSearchCV(estimator=rf, param_distributions=param_dist,
                                   n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)

rf_random_uts.fit(train_uts.drop("UTS(Mpa)", axis=1), train_uts["UTS(Mpa)"])
best_params_uts = rf_random_uts.best_params_

print("Best hyperparameters:", best_params_uts)

best_model = rf_random_uts.best_estimator_
best_score = rf_random_uts.best_score_
print("Best score:", best_score)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best hyperparameters: {'n_estimators': 100, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': None, 'bootstrap': False}
Best score: 0.821695169518179


In [20]:
# Perform RandomizedSearch for ELON
rf_random_elon = RandomizedSearchCV(estimator=rf, param_distributions=param_dist,
                                    n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)

rf_random_elon.fit(train_elon.drop("%EL", axis=1), train_elon["%EL"])

best_params_elon = rf_random_elon.best_params_

print("Best hyperparameters:", best_params_elon)

best_model = rf_random_elon.best_estimator_
best_score = rf_random_elon.best_score_
print("Best score:", best_score)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best hyperparameters: {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 10, 'bootstrap': False}
Best score: 0.6779151369104409


##Question 4 - Answer (Final implement of the problem)

In [21]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize

# Load the dataset from Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

data = pd.read_excel("/content/gdrive/MyDrive/TIES5830/Metal_Data.xlsx", header=0)
data = data[["Nb", "Ti", "V", "C", "Si", "Mn", "P", "S", "Mo", "Ni", "Al", "N", "B", "Cr", "Ce", "Cu", "Ca", "Zr", "O", "Sn", "YS(Mpa)", "UTS(Mpa)", "%EL"]]

# Alloying elements to be used as features
alloying_elements = ["Nb", "Ti", "V", "C", "Si", "Mn", "P", "S", "Mo", "Ni", "Al", "N", "B", "Cr", "Ce", "Cu", "Ca", "Zr", "O", "Sn"]

# Split the dataset into subsets for each objective
df_ys = data[data["YS(Mpa)"].notna()]
selected_columns_ys = alloying_elements + ["YS(Mpa)"]
df_ys = df_ys[selected_columns_ys]

df_uts = data[data["UTS(Mpa)"].notna()]
selected_columns_uts = alloying_elements + ["UTS(Mpa)"]
df_uts = df_uts[selected_columns_uts]

df_elon = data[data["%EL"].notna()]
selected_columns_elon = alloying_elements + ["%EL"]
df_elon = df_elon[selected_columns_elon]

# Split into training and testing sets
train_ys, test_ys = train_test_split(df_ys, test_size=0.2, random_state=42)
train_uts, test_uts = train_test_split(df_uts, test_size=0.2, random_state=42)
train_elon, test_elon = train_test_split(df_elon, test_size=0.2, random_state=42)

# Build Random Forest Regressors with specified hyperparameters
rf_ys = RandomForestRegressor(
    n_estimators=200,
    min_samples_split=4,
    min_samples_leaf=1,
    max_features='sqrt',
    max_depth=None,
    bootstrap=True
)

rf_uts = RandomForestRegressor(
    n_estimators=100,
    min_samples_split=4,
    min_samples_leaf=2,
    max_features='log2',
    max_depth=None,
    bootstrap=False
)

rf_elon = RandomForestRegressor(
    n_estimators=200,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='sqrt',
    max_depth=10,
    bootstrap=False
)

# Train the models for each objective
rf_ys.fit(train_ys.drop("YS(Mpa)", axis=1), train_ys["YS(Mpa)"])
rf_uts.fit(train_uts.drop("UTS(Mpa)", axis=1), train_uts["UTS(Mpa)"])
rf_elon.fit(train_elon.drop("%EL", axis=1), train_elon["%EL"])

# Define functions to predict objective values with valid feature names
def ys_function(x):
    df_x = pd.DataFrame([x], columns=alloying_elements)  # Ensure valid feature names
    return rf_ys.predict(df_x)[0]  # Predict yield strength

def uts_function(x):
    df_x = pd.DataFrame([x], columns=alloying_elements)  # Ensure valid feature names
    return rf_uts.predict(df_x)[0]  # Predict ultimate tensile strength

def elon_function(x):
    df_x = pd.DataFrame([x], columns=alloying_elements)  # Ensure valid feature names
    return rf_elon.predict(df_x)[0]  # Predict percentage elongation

# Define a reference point to guide optimization
reference_point = [400, 500, 20]

# Calculate the predicted objective values
def scalar_objective(x):
    ys = ys_function(x)
    uts = uts_function(x)
    elon = elon_function(x)

    # Calculate the Euclidean distance from the reference point
    dist = np.linalg.norm([ys - reference_point[0], uts - reference_point[1], elon - reference_point[2]])

    return dist

# Define bounds based on alloying elements
bounds = [(data[col].min(), data[col].max()) for col in alloying_elements]

# Define an initial guess for the optimization
initial_guess = [((b[0] + b[1]) / 2) for b in bounds]

# Solve the optimization problem with the scalar objective function
result = minimize(scalar_objective, initial_guess, bounds=bounds, method='SLSQP')

# Retrieve the optimal solution and the corresponding objective values
optimal_solution = result.x  # Optimal decision variables
optimal_ys = ys_function(optimal_solution)  # Predicted yield strength
optimal_uts = uts_function(optimal_solution)  # Predicted ultimate tensile strength
optimal_elon = elon_function(optimal_solution)  # Predicted percentage elongation

# Display the optimal solution and the predicted objective values
print("Optimal alloy composition:", optimal_solution)
print("Predicted yield strength (MPa):", optimal_ys)
print("Predicted ultimate tensile strength (MPa):", optimal_uts)
print("Predicted percentage elongation:", optimal_elon)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Optimal alloy composition: [2.25000000e-01 1.55000000e-01 4.00000000e-01 5.15600000e-01
 3.00000000e-01 8.27500000e-01 3.40000000e-01 2.09794894e-02
 5.00000000e-01 2.02500000e+00 9.50000000e-02 3.50000000e-02
 2.10000000e-03 8.30000000e-01 9.99023306e-03 1.80000000e-01
 9.50000000e-04 3.99609322e-03 4.79531187e-03 0.00000000e+00]
Predicted yield strength (MPa): 536.3232316774892
Predicted ultimate tensile strength (MPa): 897.4888333333332
Predicted percentage elongation: 20.96589460784314
